# 데이터 전처리

In [5]:
import json

# 1. JSON 파일 열기
with open("./data/filtered_case_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. 판례내용이 비어있는 항목과 비어있지 않은 항목 나누기
non_empty_case_contents = [case for case in data if case.get("판례내용")]

# 3. 새 파일로 저장
with open("./data/filtered_case_data_non_empty.json", "w", encoding="utf-8") as f:
    json.dump(non_empty_case_contents, f, ensure_ascii=False, indent=2)

# 4. 확인 출력
print(f"전체 판례 수: {len(data)}")
print(f"판례내용이 비어있지 않은 항목 수: {len(non_empty_case_contents)}")
print("✅ filtered_case_data_non_empty.json 파일 저장 완료!")


전체 판례 수: 9654
판례내용이 비어있는 항목 수: 4105
판례내용 항목 수: 5549


In [11]:
import json
import re

# 항목 분리 함수 정의
def split_articles(text):
    matches = list(re.finditer(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩)', text))
    if not matches:
        return {"제1항": text.strip()}

    result = {}
    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        content = text[start + 1:end].strip()
        result[f"제{i + 1}항"] = content
    return result

# 파일 열기
with open("./data/cleaned_law_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

processed_data = {}

# 법률명 단위로 반복
for law_name, articles in raw_data.items():
    processed_articles = []
    for article in articles:
        content = article.get("조항내용", "")
        article["조항내용"] = split_articles(content)
        processed_articles.append(article)
    processed_data[law_name] = processed_articles

# 저장
with open("./data/cleaned_law_data_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

print("✅ 조문 항별 분할 완료! cleaned_law_data_preprocessed.json 에 저장됨.")


✅ 조문 항별 분할 완료! cleaned_law_data_preprocessed.json 에 저장됨.


In [14]:
!pip install langchain langchain-community langchain-openai chromadb gradio tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.7 MB/s eta 0:00:00a 0:00:01


In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 26.6 MB/s eta 0:00:00a 0:00:01


In [ ]:
import json
import os
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


# Load and split law data
def load_and_split_law_data(path="data/cleaned_law_data_preprocessed.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    docs = []
    for law_name, articles in data.items():
        for article in articles:
            조문번호 = article["조항번호"]
            제목 = article["조항제목"]
            조문내용 = article["조항내용"]
            for 항, 내용 in 조문내용.items():
                full_text = f"[{law_name} {조문번호} {항}] {제목}\n{내용}"
                docs.append(Document(page_content=full_text, metadata={
                    "source": f"{law_name} {조문번호} {항}"
                }))
    return docs

# Load and split case data
def load_and_split_case_data(path="data/filtered_case_data.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    docs = []
    for case in data:
        if case.get("판례내용"):
            content = json.dumps(case["판례내용"], ensure_ascii=False, indent=2)
            title = case["사건명"]
            metadata = {
                "사건번호": case["사건번호"],
                "선고일자": case["선고일자"],
                "사건명": case["사건명"]
            }
            docs.append(Document(page_content=f"[판례] {title}\n{content}", metadata=metadata))
    return docs

# Build vector DB (only once)
def build_vectorstore():
    law_docs = load_and_split_law_data()
    case_docs = load_and_split_case_data()
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    all_docs = splitter.split_documents(law_docs + case_docs)
    embeddings = OpenAIEmbeddings()
    vectordb = FAISS.from_documents(all_docs, embedding=embeddings)
    vectordb.save_local("law_case_db")
    return vectordb

# Check if FAISS DB exists
def initialize_vector_db():
    if not os.path.exists("law_case_db/index.faiss"):
        print("📦 벡터 DB가 없어 생성합니다...")
        return build_vectorstore()
    print("✅ 벡터 DB 로딩 중...")
    embeddings = OpenAIEmbeddings()
    return FAISS.load_local("law_case_db", embeddings, allow_dangerous_deserialization=True)

# Initialize vector DB
vectordb = initialize_vector_db()
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini"),
    retriever=retriever,
    return_source_documents=True
)

# Gradio UI
def answer_query(query):
    result = qa_chain(query)
    sources = "\n\n".join([
        f"🔹 {doc.metadata.get('source', '미상')}\n{doc.page_content[:300]}..."
        for doc in result["source_documents"]
    ])
    return result["result"], sources

demo = gr.Interface(
    fn=answer_query,
    inputs=gr.Textbox(lines=2, placeholder="예: 혼인무효 확인소송은 이혼 후에도 제기 가능한가요?"),
    outputs=["text", "text"],
    title="📚 법률+판례 RAG 검색기",
    description="자연어로 질문하면 관련 법조문과 판례를 검색하여 설명해줍니다."
)

if __name__ == "__main__":
    demo.launch()


📦 벡터 DB가 없어 생성합니다...
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/var/folders/st/bf749bkd2lv_11d1rlyjcgz00000gn/T/ipykernel_34480/906641993.py:78: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain(query)


In [ ]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


# ✅ FAISS 벡터 DB 로딩 함수
def initialize_vector_db():
    if not os.path.exists("law_case_db/index.faiss"):
        raise FileNotFoundError("❌ FAISS DB 파일이 존재하지 않습니다. 먼저 build_vectorstore()를 실행해주세요.")
    print("✅ FAISS 벡터 DB 로딩 중...")
    embeddings = OpenAIEmbeddings()
    return FAISS.load_local("law_case_db", embeddings, allow_dangerous_deserialization=True)

# ✅ 프롬프트 템플릿 정의 (가족법 전문 AI 상담사 역할)
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
당신은 '가족법 전문 AI 상담사'입니다. 아래 사용자의 질문에 대해 제공된 문서 내용(법률 조문 및 판례)에 근거하여 정확하고 신중하게 답변해 주세요.

❓질문:
{question}

📄참고 문서:
{context}

💬답변:
- 위 문서의 내용에 기반하여 법적 해석 또는 설명을 제공하세요.
- 명확한 표현을 사용하고, 문서에 없는 내용은 추측하지 마세요.
- 법률 용어는 필요한 경우 간단히 풀이해 주세요.
"""
)

# ✅ DB 및 체인 초기화
vectordb = initialize_vector_db()
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini"),
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# ✅ 사용자 질문 응답 함수
def answer_query(query):
    query = query.strip()
    if not query:
        return "❗질문을 입력해주세요.", ""

    result = qa_chain(query)
    sources = "\n\n".join([
        f"🔹 {doc.metadata.get('source', '미상')}\n{doc.page_content[:300]}..."
        for doc in result.get("source_documents", [])
    ])
    return result.get("result", "답변 생성 중 오류가 발생했습니다."), sources

# ✅ Gradio UI 구성
demo = gr.Interface(
    fn=answer_query,
    inputs=gr.Textbox(lines=2, placeholder="예: 협의이혼으로 위자료를 받았으면 사해행위가 될 수 없나요?"),
    outputs=["text", "text"],
    title="📚 가족법 해결사 - 이혼 & 양육권 상담 AI (LangSmith 추적 포함)",
    description="자연어로 질문하면 관련 법조문과 판례를 검색하여 설명해줍니다."
)

# ✅ 앱 실행
if __name__ == "__main__":
    demo.launch()


✅ FAISS 벡터 DB 로딩 중...
Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
